In [1]:
# Standard library imports
import json
from typing import List, Optional

# Third-party imports
from dotenv import load_dotenv
import tiktoken
from pprint import pprint

# llama-index imports
from llama_index.agent.openai import OpenAIAssistantAgent, OpenAIAgent
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool, ToolMetadata
from llama_index.llms.openai import OpenAI

# Local imports
from ai_config_schema import LLM
from get_llm import get_llm, token_counter, get_llm_azure
from web_search import (
    get_page_content,
    process_search_results,
    generate_queries_search_engine,
)
from llama_index.core.agent.react import ReActChatFormatter


c:\Users\phi.nguyen\.conda\envs\neurond\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.core.agent.react.base import ReActAgent
from llama_index.core.agent.react.output_parser import ReActOutputParser
from llama_index.core.base.llms.types import ChatResponse
from llama_index.core.agent.react.types import ResponseReasoningStep
from llama_index.core.agent.react.formatter import ReActChatFormatter

In [11]:
class CustomReActOutputParser(ReActOutputParser):
    def parse(self, output: str, is_streaming: bool = False) -> ResponseReasoningStep:
        if "Action:" in output:
            return self.parse_action_reasoning_step(output)

        if "Answer:" in output:
            thought, answer = self.extract_final_response(output)
            return ResponseReasoningStep(
                thought=thought, response=answer, is_streaming=is_streaming
            )

        if "Thought:" not in output:
            return ResponseReasoningStep(
                thought="(Implicit) I can answer without any more tools!",
                response=output,
                is_streaming=True,  # Allow streaming for direct responses
            )

        raise ValueError(f"Could not parse output: {output}")


class CustomReActAgent(ReActAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.output_parser = CustomReActOutputParser()

    def _infer_stream_chunk_is_final(
        self, chunk: ChatResponse, missed_chunks_storage: list
    ) -> bool:
        latest_content = chunk.message.content
        if latest_content:
            if "Action:" in latest_content:
                return False  # Do not finalize if an action is detected
            if "Answer:" in latest_content:
                missed_chunks_storage.clear()
                return True
            if "Thought" not in latest_content:
                return True
        return False


# Usage
# Initialize your custom agent with the necessary parameters
# custom_agent = CustomReActAgent(tools=your_tools, llm=your_llm, memory=your_memory, ...)


# Agent with web tool


In [16]:
load_dotenv()


def run_agent(
    model_name: str,
    system_prompt: str,
    tools: Optional[List] = None,
    history: Optional[List] = None,
):
    if model_name in ["gpt-4o-mini", "gpt-4-turbo", "gpt-4"]:
        agent = CustomReActAgent.from_tools(
            tools=tools,
            system_prompt=system_prompt,
            chat_history=history,
            llm=get_llm_azure(),
            verbose=False,
            max_iterations=10,
            react_chat_formatter=ReActChatFormatter(context=system_prompt),
        )
    else:
        agent = CustomReActAgent.from_tools(
            tools=tools,
            system_prompt=system_prompt,
            chat_history=history,
            llm=get_llm(model_name),
            verbose=False,
            max_iterations=10,
            react_chat_formatter=ReActChatFormatter(context=system_prompt),
        )

    return agent

In [17]:
from system_prompt import DEFAULT_SYSTEM_PROMPT, DEFAULT_SYSTEM_PROMPT_OPTIMIZED
import datetime

DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
DEFAULT_SYSTEM_PROMPT_OPTIMIZED_WITH_TIME = DEFAULT_SYSTEM_PROMPT_OPTIMIZED.format(
    date=str(datetime.datetime.now().date())
)

In [18]:
# Configure tools
web_search_tool = FunctionTool.from_defaults(
    fn=process_search_results,
    tool_metadata=ToolMetadata(
        name="web_search_tool",
        description=(
            f"Used to retrieve information about up-to-date information, website or the information out of LLM's knowledge.'"
        ),
    ),
)

web_fetch_tool = FunctionTool.from_defaults(
    fn=get_page_content,
    tool_metadata=ToolMetadata(
        name="web_fetch_tool",
        description=(f"Used to fetch information of specific url"),
    ),
)

tools = [web_fetch_tool, web_search_tool]

In [19]:
DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
model_name = "claude-medium"
agent = run_agent(
    model_name=model_name,
    system_prompt=DEFAULT_SYSTEM_PROMPT_WITH_TIME,
    tools=tools,
    history="",
)

input = "kết quả trận T1 vs BLG mới nhất"

resp = agent.stream_chat(input)

for token in resp.response_gen:
    print(token, end="")


sub question: ['kết quả trận T1 vs BLG mới nhất']
Kết quả trận đấu gần đây nhất giữa T1 và BLG là T1 đã giành chiến thắng với tỷ số 2-1 trong trận tứ kết của giải đấu Esports World Cup 2024. Đây là một trận đấu khó khăn và kịch tính cho cả hai đội. T1 đã xuất sắc đánh bại BLG và trở thành đội đầu tiên giành vé vào bán kết của giải đấu. Trận đấu diễn ra rất căng thẳng với T1 thắng ván 1, BLG thắng ván 2, và T1 cuối cùng đã giành chiến thắng ở ván quyết định. Đây được xem là màn phục thù thành công của T1 sau thất bại trước BLG tại MSI 2024 trước đó.

In [20]:
input = "tôi vừa hỏi gì?"
resp = agent.stream_chat(input)
for token in resp.response_gen:
    print(token, end="")


Bạn vừa hỏi về "kết quả trận T1 vs BLG mới nhất". Cụ thể, bạn muốn biết kết quả của trận đấu gần đây nhất giữa hai đội T1 và BLG trong một giải đấu Esports.

In [21]:
input = "Tổng bí thư là ai??"
resp = agent.stream_chat(input)
for token in resp.response_gen:
    print(token, end="")


sub question: ['Tổng bí thư Việt Nam hiện nay là ai']
Theo thông tin mới nhất, Tổng Bí thư Ban Chấp hành Trung ương Đảng Cộng sản Việt Nam hiện nay là đồng chí Tô Lâm. Ông được bầu làm Tổng Bí thư tại Hội nghị Trung ương khóa XIII bất thường vào ngày 3 tháng 8 năm 2024, thay thế đồng chí Nguyễn Phú Trọng đã mất khi đang tại nhiệm. Đồng chí Tô Lâm là Đại tướng Công an nhân dân Việt Nam và trước đó đã từng giữ chức vụ Chủ tịch nước từ tháng 5 đến tháng 10 năm 2024.

In [22]:
input = "Thế còn chủ tịch nước?"
resp = agent.stream_chat(input)
for token in resp.response_gen:
    print(token, end="")


TTôi xin lỗi, tôi đã cung cấp thông tin không chính xác trong câu trả lời trước. Tôi sẽ kiểm tra lại thông tin mới nhất về Chủ tịch nước Việt Nam.

Thought: Tôi cần tìm kiếm thông tin mới nhất về Chủ tịch nước Việt Nam hiện tại. Tôi sẽ sử dụng công cụ tìm kiếm web để có thông tin chính xác và cập nhật.

Action: web_search_tool
Action Input: {"input": "Chủ tịch nước Việt Nam hiện nay"}